# ETL para carregar o PIB, População e o Centróide dos municípios
**Versão adaptada para ambiente local**

In [ ]:
# Versão local - removida autenticação do Google Colab
import pandas as pd
import os
import subprocess
import sys
print('Ambiente local configurado!')

## Pré-processamento dos centroides dos municipios

### Solução 1 - Processamento do arquivo de banco de dados (dbf)

In [ ]:
# Instalar biblioteca dbf se necessário
try:
    import dbf
    print("Biblioteca 'dbf' já está instalada")
except ImportError:
    print("Instalando biblioteca 'dbf'...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "dbf"])
    import dbf
    print("Biblioteca 'dbf' instalada com sucesso!")

In [ ]:
# Download do arquivo de dados (equivalente ao !wget)
import urllib.request
import os

url = "https://github.com/renatocol/Latitude_Longitude_Brasil/raw/master/BR_Localidades_2010.dbf"
filename = "BR_Localidades_2010.dbf"

if not os.path.exists(filename):
    print(f"Baixando {filename}...")
    urllib.request.urlretrieve(url, filename)
    print(f"Download concluído: {filename}")
else:
    print(f"Arquivo {filename} já existe")

# Listar arquivos no diretório atual (equivalente ao !ls)
print("\nArquivos no diretório atual:")
for item in os.listdir('.'):
    if os.path.isfile(item):
        size = os.path.getsize(item)
        print(f"{item} ({size:,} bytes)")
    else:
        print(f"{item}/ (diretório)")

In [ ]:
import dbf
import pandas as pd

table = dbf.Table(filename='./BR_Localidades_2010.dbf')
table.open(dbf.READ_ONLY)
df = pd.DataFrame(table)
table.close()

print(df)

In [ ]:
df.head()

In [ ]:
# Equivalente SQL: select col9 as cod_ibge, col16 as categoria, col18 as long, col19 as lat from df
df_geo = df[[9,16,18,19]].rename(columns={9:"cod_ibge", 16:"categoria", 18:"long", 19:"lat"})
df_geo['cod_ibge'] = df_geo['cod_ibge'].str.strip()
df_geo['categoria'] = df_geo['categoria'].str.strip()
df_geo = df_geo[df_geo['categoria']=='CIDADE']
df_geo.info()

In [ ]:
df_geo['lat_long'] = df_geo[['lat','long']].apply(lambda x: f"{str(x['lat']).replace(',','.')},{str(x['long']).replace(',','.')}", axis=1)
df_geo.head()

In [ ]:
cat = 'CIDADE      '
cat.strip()

### Solução 2 - Converter o arquivo dbf para xlsx usando o excel. Processar o arquivo xlsx com o pandas
#### Vantagem: mais rápido do que o pandas processar o arquivo dbf.
#### URL do arquivo exportado para xlsx https://github.com/alexlopespereira/mba_enap/raw/refs/heads/main/data/originais/centroide_municipios/BR_Localidades_2010_v1.xlsx

In [ ]:
# Use a biblioteca pandas.
import pandas as pd

# Instalar openpyxl se necessário para ler arquivos Excel
try:
    import openpyxl
except ImportError:
    print("Instalando openpyxl para leitura de arquivos Excel...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "openpyxl"])
    import openpyxl

# Leia o arquivo Excel disponível em: # https://github.com/alexlopespereira/mba_enap/raw/refs/heads/main/data/originais/centroide_municipios/BR_Localidades_2010_v1.xlsx
# Garanta que a coluna "CD_GEOCODM,C,20" seja importada como texto (str).
df_xlsx = pd.read_excel('https://github.com/alexlopespereira/mba_enap/raw/refs/heads/main/data/originais/centroide_municipios/BR_Localidades_2010_v1.xlsx', dtype={'CD_GEOCODM,C,20':str})
# Mostre as 5 primeiras linhas do dataframe.
df_xlsx.head()

In [ ]:
df_xlsx.columns

In [ ]:
# Crie um novo dataframe chamado dfxlsx_geo contendo apenas as colunas:
# "CD_GEOCODM,C,20" → renomeada para "cod_ibge"
# "NM_CATEGOR,C,50" → renomeada para "categoria"
# "LONG,N,24,6" → renomeada para "long"
# "LAT,N,24,6" → renomeada para "lat"

dfxlsx_geo = df_xlsx[['CD_GEOCODM,C,20','NM_CATEGOR,C,50','LONG,N,24,6','LAT,N,24,6']].rename(columns={'CD_GEOCODM,C,20':"cod_ibge", 'NM_CATEGOR,C,50':"categoria", 'LONG,N,24,6':"long", 'LAT,N,24,6':"lat"})
dfxlsx_geo.head()

In [ ]:
# Filtrar apenas cidades
dfxlsx_geo_cidades = dfxlsx_geo[dfxlsx_geo['categoria'] == 'CIDADE'].copy()
print(f"Total de cidades: {len(dfxlsx_geo_cidades)}")
dfxlsx_geo_cidades.head()

In [ ]:
# Criar coluna lat_long
dfxlsx_geo_cidades['lat_long'] = dfxlsx_geo_cidades[['lat','long']].apply(lambda x: f"{str(x['lat']).replace(',','.')},{str(x['long']).replace(',','.')}", axis=1)
dfxlsx_geo_cidades.head()

In [ ]:
# Informações sobre o dataset final
print("Informações do dataset de cidades:")
dfxlsx_geo_cidades.info()
print("\nEstatísticas descritivas:")
dfxlsx_geo_cidades.describe()

In [ ]:
# Salvar resultado em arquivo CSV para uso posterior
output_file = 'municipios_coordenadas.csv'
dfxlsx_geo_cidades.to_csv(output_file, index=False)
print(f"Dados salvos em: {output_file}")
print(f"Arquivo contém {len(dfxlsx_geo_cidades)} registros de municípios brasileiros")